## Code free Autogluon test

Code free Autogluon을 실행합니다.  
Autogluon에서 제공되는 데이터를 이용하기 위해 autogloun을 설치합니다.

In [3]:
!pip install autogluon

     |████████████████████████████████| 622 kB 34.3 MB/s eta 0:00:01
     |████████████████████████████████| 66.1 MB 78.2 MB/s eta 0:00:01
  Using cached pyarrow-1.0.0-cp36-cp36m-manylinux2014_x86_64.whl (17.2 MB)
     |████████████████████████████████| 101 kB 25.0 MB/s ta 0:00:01
     |████████████████████████████████| 810 kB 91.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 83.2 MB/s eta 0:00:01
     |████████████████████████████████| 882 kB 95.8 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 105.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 95.0 MB/s eta 0:00:01
     |████████████████████████████████| 28.6 MB 72.9 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 105.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 76.5 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 8.8 MB/s s eta 0:00:01
     |████████████████████

In [54]:
import pandas as pd
import autogluon as ag
from autogluon import TabularPrediction as task

데이터셋을 다운로드합니다.

In [9]:
train_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
test_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv | Columns = 15 / 15 | Rows = 39073 -> 39073
Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


In [10]:
print(train_data.shape)
print(test_data.shape)

(39073, 15)
(9769, 15)


목표변수의 컬럼명을 `target` 으로 수정하고 저장합니다.

In [35]:
columns = list(train_data.columns[:-1])
columns.append('target')
train_data.columns = columns
test_data.columns = columns

income 이 50K이상인지 이하인지 구별하는 데이터입니다.

In [36]:
train_data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,25,Private,178478,Bachelors,13,Never-married,Tech-support,Own-child,White,Female,0,0,40,United-States,<=50K
1,23,State-gov,61743,5th-6th,3,Never-married,Transport-moving,Not-in-family,White,Male,0,0,35,United-States,<=50K
2,46,Private,376789,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,15,United-States,<=50K
3,55,?,200235,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,50,United-States,>50K
4,36,Private,224541,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,El-Salvador,<=50K


In [37]:
test_data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States,<=50K
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States,<=50K
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States,>50K
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States,<=50K
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States,<=50K


In [38]:
train_data.to_csv('income_train.csv')
test_data.to_csv('income_test.csv')


CloudFormation에서 생성한 S3 경로에 파일을 복사합니다. 

In [59]:
yourname = '<Your Name>'

In [60]:
!aws s3 ls s3://code-free-automl-us-east-1-{yourname}/data/

2020-11-13 03:44:57          0 
2020-11-14 07:26:36    1191233 income_test.csv
2020-11-14 07:26:35    4797088 income_train.csv


In [39]:
!aws s3 cp income_train.csv s3://code-free-automl-us-east-1-{yourname}/data/income_train.csv
!aws s3 cp income_test.csv s3://code-free-automl-us-east-1-{yourname}/income_test.csv

upload: ./income_train.csv to s3://code-free-automl-us-east-1-leonkang/data/income_train.csv
upload: ./income_test.csv to s3://code-free-automl-us-east-1-leonkang/data/income_test.csv


Lambda와 SageMaker training 작업이 실행되었을 것입니다. 작업은 10분정도 걸립니다. SageMaker Training job이 completed 상태가 되는 것을 기다리면서 다음을 확인합니다. 
- Lambda 콘솔에서 Lambda 로그를 확인합니다.  
- SageMaker 콘솔에서 Training Job 로그를 확인합니다.

SageMaker Training job이 completed 상태가 된 후 S3 results 경로를 확인하면 Training job 이름을 가지는 폴더가 생성되었을 것입니다.

In [61]:
!aws s3 ls s3://code-free-automl-us-east-1-{yourname}/results/

                           PRE income-autogluon-2020-11-14-07-26-36-439/


해당 폴더의 파일을 복사하여 가져옵니다.

In [46]:
!aws s3 cp s3://code-free-automl-us-east-1-{yourname}/results/income-autogluon-2020-11-14-07-26-36-439/ ./

fatal error: An error occurred (404) when calling the HeadObject operation: Key "results/income-autogluon-2020-11-14-07-26-36-439/" does not exist


In [52]:
!aws s3 cp s3://code-free-automl-us-east-1-{yourname}/results/income-autogluon-2020-11-14-07-26-36-439/income_leaderboard.csv income_leaderboard.csv
!aws s3 cp s3://code-free-automl-us-east-1-{yourname}/results/income-autogluon-2020-11-14-07-26-36-439/income_test_predictions.csv income_test_predictions.csv
    

download: s3://code-free-automl-us-east-1-leonkang/results/income-autogluon-2020-11-14-07-26-36-439/income_leaderboard.csv to ./income_leaderboard.csv
download: s3://code-free-automl-us-east-1-leonkang/results/income-autogluon-2020-11-14-07-26-36-439/income_test_predictions.csv to ./income_test_predictions.csv


In [55]:
leaderboard = pd.read_csv('income_leaderboard.csv')
test_predictions = pd.read_csv('income_test_predictions.csv')

leaderboard 파일에 automl실행 결과가 저장되어 있습니다.

In [58]:
leaderboard

,Unnamed: 0,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,0,weighted_ensemble_k0_l1,0.8876,0.739162,22.901342,0.005568,1.172889,1,True,12
1,1,CatboostClassifier,0.8848,0.038987,7.931631,0.038987,7.931631,0,True,9
2,2,LightGBMClassifier,0.8808,0.024949,0.474675,0.024949,0.474675,0,True,7
3,3,LightGBMClassifierCustom,0.8808,0.029646,0.934098,0.029646,0.934098,0,True,11
4,4,LightGBMClassifierXT,0.8756,0.029975,0.593730,0.029975,0.593730,0,True,8
5,5,NeuralNetClassifier,0.8692,0.349836,176.660435,0.349836,176.660435,0,True,10
6,6,RandomForestClassifierGini,0.8648,0.213521,4.885556,0.213521,4.885556,0,True,1
7,7,RandomForestClassifierEntr,0.8644,0.213355,6.861832,0.213355,6.861832,0,True,2
8,8,ExtraTreesClassifierGini,0.8576,0.213429,3.449850,0.213429,3.449850,0,True,3
9,9,ExtraTreesClassifierEntr,0.8540,0.213061,4.052643,0.213061,4.052643,0,True,4


test_predictions 파일에 테스트데이터에 대한 예측결과가 저장되어 있습니다.

In [57]:
test_predictions.head()

,Predicted
0,<=50K
1,<=50K
2,>50K
3,<=50K
4,<=50K
